<a href="https://colab.research.google.com/github/erinmcmahon26/MNIST-Digit-Recognizer/blob/main/Digit_Recognizer_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import datetime
from packaging import version
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics

from collections import Counter
import numpy as np
import pandas as pd

In [3]:
#!pip install tensorflow

In [5]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist

In [ ]:
!wget https://github.com/erinmcmahon26/MNIST-Digit-Recognizer-Kaggle-Competition/archive/refs/heads/main.zip 

In [ ]:
!unzip "/content/main.zip" -d "/content/output_folder/"

In [ ]:
!unzip "/content/output_folder/MNIST-Digit-Recognizer-main/test.csv.zip" -d "/content/output_folder/MNIST-Digit-Recognizer-main/csvs"

In [ ]:
df_train = pd.read_csv('/content/output_folder/MNIST-Digit-Recognizer-main/csvs/train.csv')

In [ ]:
df_test = pd.read_csv('/content/output_folder/MNIST-Digit-Recognizer-main/csvs/test.csv')

In [ ]:
y = df_train['label'].values
x = df_train.drop(columns = ['label']).values

In [ ]:
x_test = df_test

In [ ]:
x = x/255
x_test = x_test/255